In [114]:
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt 
import numpy as np
import torchvision
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision import datasets, models, transforms 
from torch.optim import lr_scheduler
import time 
import os 
import copy 
import torch.optim as optim 
import torch.utils
import sys
import PIL
from torch.utils.tensorboard import SummaryWriter

In [116]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [117]:
mean = np.array([0.485,0.456,0.406])
std = np.array([0.229,0.224,0.225])

In [118]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
}

In [119]:
data_dir = 'C:/Users/user/Asztali gép/liminal space dataset'
sets = ['train','val']

In [131]:
class LiminalSpacesDataset(Dataset):
    def __init__(self,root_dir,transform=data_transforms['train']):
        self.root_dir = root_dir
        self.transform = transform
        self.files = os.listdir(root_dir)
    def __len__(self):
        return len(self.files)

    def __getitem__(self,index):
        image_path = os.path.join(self.root_dir,self.files[index])
        image = Image.open(image_path).convert("RGB")
        y_label = torch.tensor(int(self.files[index].split(' ')[0]))
        image = self.transform(image)
        
        return (image,y_label)

In [182]:
#hyper parameters
batchsize=32
numepochs = 50
numclasses = 10
learningRate = 1e-3

In [155]:
dataset = LiminalSpacesDataset(root_dir='preprocessed_data')
train_size = int(0.9*len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataset.transform = data_transforms['train']
test_dataset.transform = data_transforms['val']
train_loader = DataLoader(train_dataset,batch_size=batchsize,shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset,batch_size=batchsize,shuffle=False, num_workers=0)


In [175]:
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

numfeatures = model.fc.in_features
model.fc = nn.Linear(numfeatures, 1)

model = model.to(device)

criterion = nn.MSELoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
learningrateschedular = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [183]:
for epoch in range(numepochs):
    running_loss = 0
    for i,(images,scores) in enumerate(train_loader):
        scores = torch.log(scores.to(device)+1)
        images = images.to(device)
        guesses = model(images).squeeze()
        loss = criterion(guesses,scores)
        with torch.no_grad():
            running_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i%100 == 0:
            print('epoch: ',epoch, 'loss: ',loss.item())
    print((running_loss/len(train_loader)).item())    
    

epoch:  0 loss:  5.560339450836182
epoch:  0 loss:  2.864384889602661
4.864541530609131
epoch:  1 loss:  5.150148391723633
epoch:  1 loss:  4.113582134246826
4.893955707550049
epoch:  2 loss:  5.408448219299316
epoch:  2 loss:  5.544589042663574
4.863783359527588
epoch:  3 loss:  4.9845781326293945
epoch:  3 loss:  3.3263726234436035
4.88345193862915
epoch:  4 loss:  4.606037139892578
epoch:  4 loss:  4.674435615539551
4.873384952545166
epoch:  5 loss:  4.960361480712891
epoch:  5 loss:  5.111510753631592
4.8152546882629395
epoch:  6 loss:  3.0549066066741943
epoch:  6 loss:  6.544426441192627
4.835152626037598
epoch:  7 loss:  4.928103446960449
epoch:  7 loss:  5.202144622802734
4.847618579864502
epoch:  8 loss:  4.2734375
epoch:  8 loss:  3.438416004180908
4.869822978973389
epoch:  9 loss:  5.466249465942383
epoch:  9 loss:  4.213550567626953
4.770711421966553
epoch:  10 loss:  4.1941986083984375
epoch:  10 loss:  5.9270124435424805
4.909775257110596
epoch:  11 loss:  3.1846480369567

In [184]:
running_loss = 0
with torch.no_grad():
    for i,(images,scores) in enumerate(test_loader):
        scores = torch.log(scores.to(device)+1)
        images = images.to(device)
        guesses = model(images).squeeze()
        loss = criterion(guesses,scores)
        running_loss += loss
    print((running_loss/len(test_loader)).item())

5.033708572387695


In [172]:
with torch.no_grad():
    for i,(images,scores) in enumerate(test_loader):
        images = images.to(device)
        scores = scores.to(device)
        guesses = model(images).squeeze()
        print(torch.abs(torch.round(torch.exp(guesses)-1)-scores))
        break
        


tensor([2.4000e+01, 8.0000e+00, 8.5180e+03, 2.8000e+01, 2.0000e+00, 5.8000e+01,
        1.2740e+03, 1.2600e+02, 6.1700e+02, 7.6730e+03, 4.0800e+02, 1.7000e+01,
        9.0000e+00, 7.2530e+03, 3.5470e+03, 9.4000e+01, 3.6500e+02, 4.9900e+02,
        4.1540e+03, 3.0000e+00, 1.5000e+02, 6.8000e+01, 1.6000e+01, 6.3000e+01,
        3.2000e+01, 1.5000e+01, 6.7000e+01, 4.0000e+00, 7.0000e+00, 1.1000e+01,
        6.2000e+01, 2.7000e+01], device='cuda:0')


In [146]:
import time

start_time = time.time()
for i, (inputs, labels) in enumerate(train_loader):
    if i == 100:  # Test first 100 batches
        break
print("Time per batch:", (time.time() - start_time) / 100)


Time per batch: 0.014122235774993896


In [ ]:
#Time per batch: 0.47989688396453856
#preprocessed: Time per batch: 0.014122235774993896
